<a href="https://colab.research.google.com/github/totvslabs/carol-notebooks/blob/main/notebooks/ChangeUserStatusInATenant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Change User Status in a Tenant
`This notebook will ACTIVATE/INACTIVATE given users inside a tenant, showing their Ids, Logins and updated status.`
`The script will ask for the following json on execution:`

```python
{
    "authentication_config" : {
        "username": "username@totvs.com.br",
        "password": "password",
        "organization": "YourOrganization",
        "tenantName": "YourTenantName"
    },
    "script_config" : {
        "users_to_activate": [
          "userId",
          "userId",
          "userId"
        ],
        "users_to_deactivate": [
          "userId",
          "userId",
          "userId"
        ]
    }
}
```
`You need TENANT ADMIN permissions to run this script.`

#### REQUIREMENTS
`These are the packages the script needs before execution.`

In [ ]:
%%capture
!pip install --quiet pycarol
from pycarol import PwdAuth, Carol
import json, sys

#### CAROL LOGIN FUNCTIONS
`These are the functions made to login into Carol. They will be the same for all notebooks (ideally) and will use pyCarol.` 

[pyCarol reference](https://github.com/totvslabs/pyCarol)

In [ ]:
def carol_connect(username, password, organization, tenantName):
    print(f"Connecting to Carol tenant {tenantName}... ", end="\n")

    return Carol(domain=tenantName,
                auth=PwdAuth(username, password), organization=organization)

#### SCRIPT FUNCTIONS
`If the script requires more functions to execute, they will be here.`

In [ ]:
def activate_user(carolObject, userId):
    uri = f"v3/users/{userId}"

    try:
        return carolObject.call_api(uri, "PUT", data={'mdmTenantUserState': 'ACTIVE'})
    except:
        return {"error" : str(sys.exc_info()[1])}

def deactivate_user(carolObject, userId):
    uri = f"v3/users/{userId}"

    try:
        return carolObject.call_api(uri, "PUT", data={'mdmTenantUserState': 'DELETED'})
    except:
        return {"error" : str(sys.exc_info()[1])}

#### CONFIGURATION FILE
`Now you will need to upload the configuration file with the format given above.`

In [ ]:
try:
    from google.colab import files
    config_file = files.upload()
    config_json = json.loads(config_file[next(iter(config_file))].decode("utf-8"))
except:
    with open('./carol.json') as config_file:
        config_json = json.loads(config_file.read())
    config_file.close()
finally:
    config_json_print = config_json
    del config_json_print['authentication_config']['password']
    print(json.dumps(config_json_print, indent=2))

#### SCRIPT EXECUTION
`The main execution of the script will happen here.`

In [ ]:
Carol = carol_connect(
    config_json['authentication_config']['username'], 
    config_json['authentication_config']['password'], 
    config_json['authentication_config']['organization'], 
    config_json['authentication_config']['tenantName'])

for user in config_json['script_config']['users_to_deactivate']:
    response = deactivate_user(Carol, user)
    print(f'{response["mdmId"]} - {response["mdmUserLogin"]} - NEW STATUS: {response["mdmTenantUserState"]}')

for user in config_json['script_config']['users_to_activate']:
    response = activate_user(Carol, user)
    print(f'{response["mdmId"]} - {response["mdmUserLogin"]} - NEW STATUS: {response["mdmTenantUserState"]}')